In [1]:
import pandas as pd
from helpers import load_data, get_groups

In [2]:
dataset_df = load_data()
dataset_df.head()

,id,text,headline,text_type,year
0,http://www.politifact.com/arizona/statements/2...,Residents of multiple states will be asked to ...,Multiple States Have Agreed To Implement A ‘Tw...,News and blog,2016
1,http://www.politifact.com/california/statement...,"Sacramento, CA - United States Senator Dianne ...",U.S. Senator Dianne Feinstein Opposes Prop. 64...,Press release,2016
2,http://www.politifact.com/california/statement...,We should anticipate black and gray markets in...,Why you should buy a locking gasoline cap,News and blog,2017
3,http://www.politifact.com/california/statement...,As a ballot initiative calling for repeal of a...,California Gas-Tax-Hike Repeal Campaign Heats Up,News and blog,2017
4,http://www.politifact.com/california/statement...,"WASHINGTON, DC The House of Representatives t...","Rep. Chu Decries ""Heartless"" ACA Repeal Vote",Press release,2017


Prepare an output file to process using LIWC

In [3]:
data_for_liwc = dataset_df[["id", "text"]]
data_for_liwc.to_csv("./liwc/full_dataset_for_liwc.csv", index=False)

Now that you've run the above, use the generated file as input for LIWC

Then, we will take the file generated by LIWC and continue with the analysis

In [4]:
liwc_df = pd.read_csv("./liwc/full_dataset_liwc_results.csv")
liwc_df.head()

,id,Segment,WC,Analytic,Clout,Authentic,Tone,WPS,BigWords,Dic,Linguistic,function,pronoun,ppron,i,we,you,shehe,they,ipron,det,article,number,prep,auxverb,adverb,conj,negate,verb,adj,quantity,Drives,affiliation,achieve,power,Cognition,allnone,cogproc,insight,cause,...,wellness,mental,substances,sexual,food,death,need,want,acquire,lack,fulfill,fatigue,reward,risk,curiosity,allure,Perception,attention,motion,space,visual,auditory,feeling,time,focuspast,focuspresent,focusfuture,Conversation,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,QMark,Exclam,Apostro,OtherP,Emoji
0,http://www.politifact.com/arizona/statements/2...,1,481,96.43,82.60,16.18,45.23,28.29,29.31,82.12,54.68,44.70,4.99,2.29,0.00,0.00,0.00,0.00,2.29,2.70,13.93,8.52,4.78,17.05,6.65,1.25,5.20,0.21,8.73,6.03,4.99,10.40,4.57,1.04,4.99,15.59,0.62,6.24,1.04,1.66,...,0.0,0.00,0.00,0.00,0.0,0.21,0.83,0.42,0.42,0.62,0.62,0.0,0.21,0.00,0.0,1.87,6.44,0.00,0.62,5.61,0.21,0.0,0.0,4.37,1.66,2.29,2.70,0.00,0.00,0.0,0.0,0.0,14.76,3.53,6.65,0.0,0.0,1.04,3.53,0.0
1,http://www.politifact.com/california/statement...,1,184,93.86,71.47,7.87,6.28,20.44,38.59,78.80,51.09,38.04,5.43,3.26,0.54,1.09,0.00,1.09,0.54,2.17,9.24,4.35,4.35,16.30,3.80,3.26,3.80,0.54,10.33,6.52,4.35,9.24,1.63,0.54,7.07,15.22,0.54,8.70,0.54,2.72,...,0.0,0.00,1.63,0.00,0.0,0.54,0.00,0.00,0.54,0.00,0.00,0.0,0.00,1.63,0.0,4.35,7.07,0.00,1.09,5.43,1.09,0.0,0.0,3.80,3.26,1.63,0.00,0.00,0.00,0.0,0.0,0.0,14.67,4.35,3.80,0.0,0.0,1.09,5.43,0.0
2,http://www.politifact.com/california/statement...,1,339,93.61,31.80,42.71,20.23,16.95,25.37,79.35,58.11,40.41,2.95,1.47,0.29,0.29,0.29,0.00,0.29,1.47,15.04,9.14,6.78,12.39,6.19,2.36,5.60,0.88,12.68,6.49,9.73,3.83,0.59,0.00,3.24,19.17,0.29,8.55,1.18,2.36,...,0.0,0.00,1.18,0.00,0.0,0.00,0.00,0.00,1.47,0.00,0.29,0.0,0.59,0.00,0.0,2.95,9.73,0.29,2.95,5.31,2.06,0.0,0.0,6.49,5.01,4.42,1.18,0.00,0.00,0.0,0.0,0.0,18.88,6.78,5.60,0.0,0.0,1.77,4.72,0.0
3,http://www.politifact.com/california/statement...,1,433,89.52,45.52,28.63,23.14,22.79,21.02,79.21,54.27,39.72,5.54,3.23,0.00,0.00,0.00,0.92,2.08,2.31,11.09,6.70,3.23,14.32,7.39,1.85,4.39,1.85,14.78,6.47,4.39,5.77,0.92,1.39,3.70,21.25,0.69,8.31,0.69,1.62,...,0.0,0.00,0.00,0.00,0.0,0.00,0.92,0.23,0.92,0.00,0.23,0.0,0.23,0.00,0.0,5.31,10.85,0.00,1.62,8.08,1.15,0.0,0.0,5.54,5.77,4.16,2.77,0.00,0.00,0.0,0.0,0.0,13.63,4.39,6.47,0.0,0.0,0.00,2.77,0.0
4,http://www.politifact.com/california/statement...,1,670,89.11,56.98,18.27,22.08,23.93,28.51,83.28,57.01,44.48,6.72,1.94,0.30,0.30,0.00,0.00,1.19,4.78,15.07,6.57,4.48,14.63,5.07,2.99,6.27,0.45,10.15,5.82,6.12,8.21,1.79,2.39,4.33,17.01,0.75,8.21,1.64,1.64,...,0.0,0.75,0.15,0.75,0.3,0.00,0.90,0.00,0.30,0.15,0.00,0.0,0.30,1.49,0.3,4.78,5.07,0.30,1.34,3.13,0.30,0.0,0.0,5.07,2.69,1.79,2.39,0.15,0.15,0.0,0.0,0.0,13.73,4.48,5.37,0.0,0.0,0.30,3.58,0.0


In [5]:
# Append the new LIWC stats columns onto the original dataframe
full_df = dataset_df.set_index("id").join(liwc_df.set_index("id"))
full_df = full_df.reset_index()
full_df.head()

,id,text,headline,text_type,year,Segment,WC,Analytic,Clout,Authentic,Tone,WPS,BigWords,Dic,Linguistic,function,pronoun,ppron,i,we,you,shehe,they,ipron,det,article,number,prep,auxverb,adverb,conj,negate,verb,adj,quantity,Drives,affiliation,achieve,power,Cognition,...,wellness,mental,substances,sexual,food,death,need,want,acquire,lack,fulfill,fatigue,reward,risk,curiosity,allure,Perception,attention,motion,space,visual,auditory,feeling,time,focuspast,focuspresent,focusfuture,Conversation,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,QMark,Exclam,Apostro,OtherP,Emoji
0,http://www.politifact.com/arizona/statements/2...,Residents of multiple states will be asked to ...,Multiple States Have Agreed To Implement A ‘Tw...,News and blog,2016,1,481,96.43,82.60,16.18,45.23,28.29,29.31,82.12,54.68,44.70,4.99,2.29,0.00,0.00,0.00,0.00,2.29,2.70,13.93,8.52,4.78,17.05,6.65,1.25,5.20,0.21,8.73,6.03,4.99,10.40,4.57,1.04,4.99,15.59,...,0.0,0.00,0.00,0.00,0.0,0.21,0.83,0.42,0.42,0.62,0.62,0.0,0.21,0.00,0.0,1.87,6.44,0.00,0.62,5.61,0.21,0.0,0.0,4.37,1.66,2.29,2.70,0.00,0.00,0.0,0.0,0.0,14.76,3.53,6.65,0.0,0.0,1.04,3.53,0.0
1,http://www.politifact.com/california/statement...,"Sacramento, CA - United States Senator Dianne ...",U.S. Senator Dianne Feinstein Opposes Prop. 64...,Press release,2016,1,184,93.86,71.47,7.87,6.28,20.44,38.59,78.80,51.09,38.04,5.43,3.26,0.54,1.09,0.00,1.09,0.54,2.17,9.24,4.35,4.35,16.30,3.80,3.26,3.80,0.54,10.33,6.52,4.35,9.24,1.63,0.54,7.07,15.22,...,0.0,0.00,1.63,0.00,0.0,0.54,0.00,0.00,0.54,0.00,0.00,0.0,0.00,1.63,0.0,4.35,7.07,0.00,1.09,5.43,1.09,0.0,0.0,3.80,3.26,1.63,0.00,0.00,0.00,0.0,0.0,0.0,14.67,4.35,3.80,0.0,0.0,1.09,5.43,0.0
2,http://www.politifact.com/california/statement...,We should anticipate black and gray markets in...,Why you should buy a locking gasoline cap,News and blog,2017,1,339,93.61,31.80,42.71,20.23,16.95,25.37,79.35,58.11,40.41,2.95,1.47,0.29,0.29,0.29,0.00,0.29,1.47,15.04,9.14,6.78,12.39,6.19,2.36,5.60,0.88,12.68,6.49,9.73,3.83,0.59,0.00,3.24,19.17,...,0.0,0.00,1.18,0.00,0.0,0.00,0.00,0.00,1.47,0.00,0.29,0.0,0.59,0.00,0.0,2.95,9.73,0.29,2.95,5.31,2.06,0.0,0.0,6.49,5.01,4.42,1.18,0.00,0.00,0.0,0.0,0.0,18.88,6.78,5.60,0.0,0.0,1.77,4.72,0.0
3,http://www.politifact.com/california/statement...,As a ballot initiative calling for repeal of a...,California Gas-Tax-Hike Repeal Campaign Heats Up,News and blog,2017,1,433,89.52,45.52,28.63,23.14,22.79,21.02,79.21,54.27,39.72,5.54,3.23,0.00,0.00,0.00,0.92,2.08,2.31,11.09,6.70,3.23,14.32,7.39,1.85,4.39,1.85,14.78,6.47,4.39,5.77,0.92,1.39,3.70,21.25,...,0.0,0.00,0.00,0.00,0.0,0.00,0.92,0.23,0.92,0.00,0.23,0.0,0.23,0.00,0.0,5.31,10.85,0.00,1.62,8.08,1.15,0.0,0.0,5.54,5.77,4.16,2.77,0.00,0.00,0.0,0.0,0.0,13.63,4.39,6.47,0.0,0.0,0.00,2.77,0.0
4,http://www.politifact.com/california/statement...,"WASHINGTON, DC The House of Representatives t...","Rep. Chu Decries ""Heartless"" ACA Repeal Vote",Press release,2017,1,670,89.11,56.98,18.27,22.08,23.93,28.51,83.28,57.01,44.48,6.72,1.94,0.30,0.30,0.00,0.00,1.19,4.78,15.07,6.57,4.48,14.63,5.07,2.99,6.27,0.45,10.15,5.82,6.12,8.21,1.79,2.39,4.33,17.01,...,0.0,0.75,0.15,0.75,0.3,0.00,0.90,0.00,0.30,0.15,0.00,0.0,0.30,1.49,0.3,4.78,5.07,0.30,1.34,3.13,0.30,0.0,0.0,5.07,2.69,1.79,2.39,0.15,0.15,0.0,0.0,0.0,13.73,4.48,5.37,0.0,0.0,0.30,3.58,0.0


Get separate dataframes for each year

In [6]:
years, years_dfs = get_groups(full_df, "year")
years_dfs[0].head()

,id,text,headline,text_type,year,Segment,WC,Analytic,Clout,Authentic,Tone,WPS,BigWords,Dic,Linguistic,function,pronoun,ppron,i,we,you,shehe,they,ipron,det,article,number,prep,auxverb,adverb,conj,negate,verb,adj,quantity,Drives,affiliation,achieve,power,Cognition,...,wellness,mental,substances,sexual,food,death,need,want,acquire,lack,fulfill,fatigue,reward,risk,curiosity,allure,Perception,attention,motion,space,visual,auditory,feeling,time,focuspast,focuspresent,focusfuture,Conversation,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,QMark,Exclam,Apostro,OtherP,Emoji
428,http://www.politifact.com/truth-o-meter/statem...,"Washington, D.C., Mar 25 - In response to sugg...",Bachmann Demands Truth: Will Obama Administrat...,Press release,2009,1,255,98.12,35.57,23.38,NaN,23.18,34.12,84.31,60.00,44.71,5.88,3.14,0.78,0.00,0.00,2.35,0.00,2.75,16.08,12.55,1.57,15.29,5.10,1.57,4.31,0.00,10.59,6.67,4.31,4.31,0.00,0.39,3.53,21.96,...,0.0,0.0,0.0,0.00,0.0,0.00,0.39,0.39,0.00,0.00,0.00,0.0,0.00,0.00,0.39,3.92,7.84,0.00,1.18,6.67,0.00,0.39,0.00,3.14,3.92,2.35,0.78,0.00,0.00,0.00,0.0,0.0,18.82,8.24,5.88,0.00,0.0,0.00,4.71,0.0
429,http://www.politifact.com/truth-o-meter/statem...,When most Americans talk about the need for he...,Taxpayer-Funded Abortion Is Not Health-Care Re...,News and blog,2009,1,814,88.12,53.26,9.62,5.82,21.42,30.10,83.29,58.35,45.95,7.49,2.70,0.12,0.00,0.00,0.61,1.23,4.79,14.86,8.35,1.35,14.86,6.88,3.69,5.04,1.11,12.29,5.41,4.91,8.60,1.11,1.35,6.27,19.53,...,0.0,0.0,0.0,2.09,0.0,0.12,1.60,0.37,0.49,0.12,0.25,0.0,0.25,0.12,0.00,3.93,5.28,0.12,0.49,4.42,0.49,0.00,0.00,3.56,2.58,4.18,3.07,0.49,0.49,0.00,0.0,0.0,20.39,5.04,4.67,0.49,0.0,2.09,8.11,0.0
430,http://www.politifact.com/truth-o-meter/statem...,A number of people in the news analysis busine...,One of these things is not like the other,News and blog,2009,1,229,74.53,13.42,57.87,1.67,20.82,23.58,87.34,64.19,52.40,6.55,3.49,1.31,0.00,0.44,0.00,1.75,3.06,11.79,6.99,1.75,16.59,9.61,6.11,5.68,3.49,15.72,6.55,2.62,3.06,0.44,0.44,2.18,28.38,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.44,0.00,0.00,0.0,0.00,3.93,0.00,5.68,9.61,0.44,0.00,6.55,0.44,1.31,0.00,3.06,7.42,2.62,0.44,0.44,0.00,0.44,0.0,0.0,11.79,4.37,6.55,0.44,0.0,0.00,0.44,0.0
431,http://www.politifact.com/truth-o-meter/statem...,Yesterday President Obama responded to my stat...,NaN,Social media,2009,1,1021,68.28,48.79,20.67,60.23,19.26,27.91,84.52,59.06,47.40,9.79,3.82,0.78,0.49,0.00,0.88,1.27,5.97,14.79,6.95,5.39,12.44,7.74,5.00,5.78,1.37,14.59,4.02,4.02,6.66,1.37,1.18,4.21,21.94,...,0.0,0.1,0.0,0.00,0.0,0.29,0.49,0.00,0.29,0.20,0.10,0.0,0.59,0.20,0.10,3.92,7.35,0.20,0.69,5.00,1.27,0.10,0.20,3.33,1.86,5.19,1.76,0.00,0.00,0.00,0.0,0.0,27.33,7.05,4.11,0.39,0.0,1.86,13.91,0.0
432,http://www.politifact.com/truth-o-meter/statem...,Secretary of Defense Robert Gates is extremely...,"Military to Pledge Oath To Obama, Not Constitu...",News and blog,2009,1,374,96.09,33.98,23.68,14.39,14.96,24.06,85.29,63.37,52.94,5.61,0.53,0.00,0.53,0.00,0.00,0.00,5.08,21.12,14.71,0.80,15.24,8.29,3.48,4.55,1.34,12.30,8.02,3.74,8.56,1.60,0.00,6.95,22.19,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.27,0.27,0.00,0.00,0.0,0.00,0.00,0.00,3.48,7.22,0.27,0.80,4.55,1.60,0.00,0.27,4.81,2.94,4.55,3.21,0.00,0.00,0.00,0.0,0.0,17.65,6.68,5.88,0.00,0.0,0.27,4.81,0.0


Get separate dataframes for each text type

In [7]:
types, types_dfs = get_groups(full_df, "text_type")
types_dfs[0].head()

,id,text,headline,text_type,year,Segment,WC,Analytic,Clout,Authentic,Tone,WPS,BigWords,Dic,Linguistic,function,pronoun,ppron,i,we,you,shehe,they,ipron,det,article,number,prep,auxverb,adverb,conj,negate,verb,adj,quantity,Drives,affiliation,achieve,power,Cognition,...,wellness,mental,substances,sexual,food,death,need,want,acquire,lack,fulfill,fatigue,reward,risk,curiosity,allure,Perception,attention,motion,space,visual,auditory,feeling,time,focuspast,focuspresent,focusfuture,Conversation,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,QMark,Exclam,Apostro,OtherP,Emoji
0,http://www.politifact.com/arizona/statements/2...,Residents of multiple states will be asked to ...,Multiple States Have Agreed To Implement A ‘Tw...,News and blog,2016,1,481,96.43,82.60,16.18,45.23,28.29,29.31,82.12,54.68,44.70,4.99,2.29,0.00,0.00,0.00,0.00,2.29,2.70,13.93,8.52,4.78,17.05,6.65,1.25,5.20,0.21,8.73,6.03,4.99,10.40,4.57,1.04,4.99,15.59,...,0.0,0.0,0.00,0.0,0.00,0.21,0.83,0.42,0.42,0.62,0.62,0.0,0.21,0.00,0.00,1.87,6.44,0.00,0.62,5.61,0.21,0.00,0.0,4.37,1.66,2.29,2.70,0.00,0.0,0.0,0.0,0.0,14.76,3.53,6.65,0.00,0.0,1.04,3.53,0.0
2,http://www.politifact.com/california/statement...,We should anticipate black and gray markets in...,Why you should buy a locking gasoline cap,News and blog,2017,1,339,93.61,31.80,42.71,20.23,16.95,25.37,79.35,58.11,40.41,2.95,1.47,0.29,0.29,0.29,0.00,0.29,1.47,15.04,9.14,6.78,12.39,6.19,2.36,5.60,0.88,12.68,6.49,9.73,3.83,0.59,0.00,3.24,19.17,...,0.0,0.0,1.18,0.0,0.00,0.00,0.00,0.00,1.47,0.00,0.29,0.0,0.59,0.00,0.00,2.95,9.73,0.29,2.95,5.31,2.06,0.00,0.0,6.49,5.01,4.42,1.18,0.00,0.0,0.0,0.0,0.0,18.88,6.78,5.60,0.00,0.0,1.77,4.72,0.0
3,http://www.politifact.com/california/statement...,As a ballot initiative calling for repeal of a...,California Gas-Tax-Hike Repeal Campaign Heats Up,News and blog,2017,1,433,89.52,45.52,28.63,23.14,22.79,21.02,79.21,54.27,39.72,5.54,3.23,0.00,0.00,0.00,0.92,2.08,2.31,11.09,6.70,3.23,14.32,7.39,1.85,4.39,1.85,14.78,6.47,4.39,5.77,0.92,1.39,3.70,21.25,...,0.0,0.0,0.00,0.0,0.00,0.00,0.92,0.23,0.92,0.00,0.23,0.0,0.23,0.00,0.00,5.31,10.85,0.00,1.62,8.08,1.15,0.00,0.0,5.54,5.77,4.16,2.77,0.00,0.0,0.0,0.0,0.0,13.63,4.39,6.47,0.00,0.0,0.00,2.77,0.0
6,http://www.politifact.com/california/statement...,"Recently, a group of special interests threate...","Repeal Californias gas tax increase, says GOP ...",News and blog,2017,1,495,95.66,65.09,5.48,10.27,21.52,32.53,83.03,60.20,46.26,6.46,2.63,0.20,1.21,0.00,0.20,1.01,3.84,14.75,9.49,2.63,15.96,4.24,3.03,5.45,0.20,9.90,7.07,6.46,12.93,4.04,3.23,5.86,12.32,...,0.0,0.0,0.00,0.0,0.20,0.20,0.00,0.00,1.41,0.20,0.00,0.0,0.81,0.81,0.81,4.85,7.07,0.61,0.61,5.05,0.61,0.00,0.4,0.61,2.42,4.04,0.40,0.00,0.0,0.0,0.0,0.0,15.56,5.05,6.67,0.20,0.0,0.00,3.64,0.0
7,http://www.politifact.com/california/statement...,"COSTA MESA, Orange County It was a surreal vi...","The pro-Russia, pro-weed, pro-Assange GOP cong...",News and blog,2017,1,1052,78.87,76.48,6.33,22.61,18.46,26.05,79.94,57.70,45.53,11.22,7.22,0.48,0.38,0.29,4.47,0.76,3.99,12.64,6.84,2.28,14.45,7.13,3.04,4.94,0.95,13.21,4.75,3.71,6.18,1.14,0.86,4.28,16.35,...,0.1,0.0,0.10,0.1,1.14,0.10,0.00,0.29,0.57,0.10,0.19,0.0,0.19,0.48,0.00,3.71,9.22,0.10,1.71,5.89,1.43,0.19,0.1,3.14,4.37,4.09,0.67,0.19,0.1,0.0,0.1,0.0,15.59,5.13,6.27,0.19,0.0,0.00,3.99,0.0


## Writing dataframes to excel spreadsheet

Save to a single Excel file with tabs for each year

In [11]:
cols_to_save = list(liwc_df.columns) + ["year"]
cols_to_save

['id',
 'Segment',
 'WC',
 'Analytic',
 'Clout',
 'Authentic',
 'Tone',
 'WPS',
 'BigWords',
 'Dic',
 'Linguistic',
 'function',
 'pronoun',
 'ppron',
 'i',
 'we',
 'you',
 'shehe',
 'they',
 'ipron',
 'det',
 'article',
 'number',
 'prep',
 'auxverb',
 'adverb',
 'conj',
 'negate',
 'verb',
 'adj',
 'quantity',
 'Drives',
 'affiliation',
 'achieve',
 'power',
 'Cognition',
 'allnone',
 'cogproc',
 'insight',
 'cause',
 'discrep',
 'tentat',
 'certitude',
 'differ',
 'memory',
 'Affect',
 'tone_pos',
 'tone_neg',
 'emotion',
 'emo_pos',
 'emo_neg',
 'emo_anx',
 'emo_anger',
 'emo_sad',
 'swear',
 'Social',
 'socbehav',
 'prosocial',
 'polite',
 'conflict',
 'moral',
 'comm',
 'socrefs',
 'family',
 'friend',
 'female',
 'male',
 'Culture',
 'politic',
 'ethnicity',
 'tech',
 'Lifestyle',
 'leisure',
 'home',
 'work',
 'money',
 'relig',
 'Physical',
 'health',
 'illness',
 'wellness',
 'mental',
 'substances',
 'sexual',
 'food',
 'death',
 'need',
 'want',
 'acquire',
 'lack',
 'fulfi

In [12]:
def save_years(writer: pd.ExcelWriter, years: list[int], dfs: list[pd.DataFrame]):
    for year, df in zip(years, dfs):
        df.to_excel(
            writer,
            sheet_name=str(year),
            index=False,
            columns=cols_to_save
        )

In [13]:
writer = pd.ExcelWriter("./output/liwc.xlsx", engine="xlsxwriter")
save_years(writer, years, years_dfs)
writer.close()

For each text type, we separate further into dataframes per year, and save the Excel files as above

In [14]:
for type, df in zip(types, types_dfs):
    years, years_dfs = get_groups(df, "year")

    type_str = str(type).lower().replace(" ", "_")

    writer = pd.ExcelWriter(f"./output/{type_str}/liwc_{type_str}.xlsx", engine="xlsxwriter")
    save_years(writer, years, years_dfs)
    writer.close()